In [0]:
# 1) Remove any stale install, then install your local package (editable)
%pip uninstall -y surveytools
%pip install -e "/Workspace/teams/survey_team/Core_automation/Correct code/surveytools" --verbose


In [0]:
# 2) Restart the Python worker in THIS notebook
dbutils.library.restartPython()


In [0]:
# 3) Confirm Python can see the installed package
import sys, importlib.metadata as im

print("Top sys.path[0..3]:")
print("\n".join(sys.path[:4]))

# Is the distribution installed into this environment?
names = [d.metadata.get("Name","").lower() for d in im.distributions()]
print("\n'surveytools' present in installed dists? ->", "surveytools" in names)

# Try importing
from surveytools.types import SummaryConfig
SummaryConfig()


In [0]:
import sys, os

# 4) Add your package's src folder to Python's search path for THIS notebook session
sys.path.insert(0, "/Workspace/teams/survey_team/Core_automation/Correct code/surveytools/src")

# 5) Now import
from surveytools.types import SummaryConfig
SummaryConfig()


In [0]:
# import bootstrap (keep at top of notebooks that use surveytools)
import sys
SRC = "/Workspace/teams/survey_team/Core_automation/Correct code/surveytools/src"
if SRC not in sys.path:
    sys.path.insert(0, SRC)


In [0]:
from surveytools.constants import ZC_TURQUOISE, ZERO_ALIASES, PRESETS
print("One color:", ZC_TURQUOISE)
print("ZERO_ALIASES count:", len(ZERO_ALIASES))
print("Preset keys:", list(PRESETS.keys()))
